In [83]:
import pandas as pd
import json

# content = pd.read_json('data/volume_control/set_by_volume.json')
# content.head()

content = json.load(open('data/volume_control/combined_volume.json'))
content

{'reduce the volume adjuster': {'intent': 'volume.decrease',
  'entities': {'volume': 'current_volume - volume_decrement'}},
 'bring down the sound output': {'intent': 'volume.decrease',
  'entities': {'volume': 'current_volume - volume_decrement'}},
 'decrease the volume output': {'intent': 'volume.decrease',
  'entities': {'volume': 'current_volume - volume_decrement'}},
 'decrease the volume adjuster level': {'intent': 'volume.decrease',
  'entities': {'volume': 'current_volume - volume_decrement'}},
 'make the volume setting quieter': {'intent': 'volume.decrease',
  'entities': {'volume': 'current_volume - volume_decrement'}},
 'soften the audio': {'intent': 'volume.decrease',
  'entities': {'volume': 'current_volume - volume_decrement'}},
 'make the volume control quieter': {'intent': 'volume.decrease',
  'entities': {'volume': 'current_volume - volume_decrement'}},
 'decrease the sound output level': {'intent': 'volume.decrease',
  'entities': {'volume': 'current_volume - volume_

In [88]:
actions, list_variables, list_num_variables = [], [], []

def entities():
    for i in content.values():
        for key, value in i['entities'].items():
            actions.append(f'{key} = {value}')


def variables():
    variables = ['current_volume', 'volume_max', 'volume_increment', 'volume_decrement', 'last_volume']
    for i in actions:
        count = 0
        var = []
        for j in variables:
            if j in i.split(' = ')[1]:
                count += 1
                var.append(j)
                
        list_variables.append(var) if var!=[] else list_variables.append(None)
        list_num_variables.append(count)


entities()
variables()

df = {
    "Utterance": [i for i in content.keys()],
    "Intent": [i['intent'] for i in content.values()],
    "Action": actions,
    # "Variables": list_variables,
    # "Num_Variables": list_num_variables
}

In [92]:
#! pip3 install openpyxl
data = pd.DataFrame(df)
data

  Using cached openpyxl-3.0.10-py2.py3-none-any.whl (242 kB)
  Using cached et_xmlfile-1.1.0-py3-none-any.whl (4.7 kB)


,Utterance,Intent,Action
0,reduce the volume adjuster,volume.decrease,volume = current_volume - volume_decrement
1,bring down the sound output,volume.decrease,volume = current_volume - volume_decrement
2,decrease the volume output,volume.decrease,volume = current_volume - volume_decrement
3,decrease the volume adjuster level,volume.decrease,volume = current_volume - volume_decrement
4,make the volume setting quieter,volume.decrease,volume = current_volume - volume_decrement
...,...,...,...
6368,turn the volume knob to 31,volume.value,volume = 31
6369,adjust the volume to 37 on the system,volume.value,volume = 37
6370,increase the volume to 82,volume.value,volume = 82
6371,adjust the sound level to 89,volume.value,volume = 89


In [93]:
data.memory_usage()

Index          128
Utterance    50984
Intent       50984
Action       50984
dtype: int64

In [94]:
data.to_excel('./data/volume_control/combined_volume.xlsx')

In [71]:
import tensorflow as tf

#tokenizer=tf.keras.preprocessing.text.Tokenizer()

tokenizer=tf.keras.layers.TextVectorization()
# tokenizer.fit_on_texts(df['Utterance'])
# df['Utterance']=tokenizer.texts_to_sequences(df['Utterance'])

In [72]:
# see all avalable columns
for i in data.columns:
    print(i)

Utterance
Intent
Action
Variables
Num_Variables


In [127]:
# print(type(data['Utterance']))
# print(data[['Utterance', 'Intent', 'Action']])

In [124]:
# tokenizer.fit_on_texts(data[['Utterance', 'Intent', 'Action']])
# data[['Utterance', 'Intent', 'Action']]=tokenizer.texts_to_sequences(data[['Utterance', 'Intent', 'Action']])

In [128]:
# Create a function to tokenize the text
# def tokenize(text):
#     tokenizer = tf.keras.preprocessing.text.Tokenizer()
#     tokenizer.fit_on_texts(text)
#     return tokenizer.texts_to_sequences(text), tokenizer
def vectorizer(text):
    vectorizer = tf.keras.layers.TextVectorization(standardize='lower_and_strip_punctuation',split='whitespace',output_mode='int')
    vectorizer.adapt(text)
   
    return vectorizer(text), vectorizer
    # tokenizer.fit_on_texts(text


data["Utterance"]= data["Utterance"].astype(str)
#data["Intent"]= data["Intent"].astype(str)
#data["Action"]= data["Action"].astype(str)
#  Tokenize the 'text' column of your DataFrame
data["Utterance"], vectorizer = vectorizer(data["Utterance"])
#data["Intent"], vectorizer = vectorizer(data["Intent"])
#data["Action"], vectorizer = vectorizer(data["Action"])

# Save the vocabulary

vocab_file = "./data/volume_control/tf_vocab.txt"
with open(vocab_file, "w") as f:
    f.write("\n".join(vectorizer.get_vocabulary()))

2023-01-28 22:54:47.693968: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


In [135]:
def vectorizer(text):
    text_vectorizer=tf.keras.layers.TextVectorization(standardize='lower_and_strip_punctuation',split='whitespace',output_mode='int')
    text_vectorizer.adapt(text)
    return text_vectorizer(text), text_vectorizer

data["Utterance"], vectorizer = vectorizer(data["Utterance"].astype(str))
vocab_file = "./data/volume_control/tf_vocab.txt"
with open(vocab_file, "w") as f:
    f.write("\n".join(vectorizer.get_vocabulary()))

2023-01-28 23:01:00.329186: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


In [136]:
data.to_dict()

{'Utterance': {0: 3,
  1: 3,
  2: 3,
  3: 3,
  4: 9,
  5: 3,
  6: 9,
  7: 3,
  8: 3,
  9: 3,
  10: 3,
  11: 5,
  12: 9,
  13: 3,
  14: 3,
  15: 5,
  16: 3,
  17: 9,
  18: 3,
  19: 3,
  20: 5,
  21: 3,
  22: 3,
  23: 3,
  24: 3,
  25: 3,
  26: 3,
  27: 3,
  28: 9,
  29: 9,
  30: 3,
  31: 3,
  32: 5,
  33: 3,
  34: 3,
  35: 3,
  36: 3,
  37: 3,
  38: 9,
  39: 3,
  40: 3,
  41: 3,
  42: 9,
  43: 9,
  44: 5,
  45: 3,
  46: 3,
  47: 3,
  48: 3,
  49: 3,
  50: 9,
  51: 3,
  52: 9,
  53: 3,
  54: 3,
  55: 9,
  56: 3,
  57: 3,
  58: 8,
  59: 3,
  60: 8,
  61: 3,
  62: 9,
  63: 3,
  64: 9,
  65: 8,
  66: 8,
  67: 3,
  68: 9,
  69: 3,
  70: 3,
  71: 3,
  72: 3,
  73: 3,
  74: 3,
  75: 8,
  76: 9,
  77: 9,
  78: 3,
  79: 8,
  80: 5,
  81: 5,
  82: 9,
  83: 3,
  84: 3,
  85: 9,
  86: 8,
  87: 3,
  88: 3,
  89: 3,
  90: 8,
  91: 3,
  92: 3,
  93: 3,
  94: 9,
  95: 3,
  96: 5,
  97: 3,
  98: 8,
  99: 8,
  100: 3,
  101: 3,
  102: 8,
  103: 8,
  104: 3,
  105: 8,
  106: 9,
  107: 3,
  108: 8,
  109: 